# LSTM Model
## Maren, Sophia & Malin - Group 4

Dies ist das etwas komplexere Model, das wir eigentlich benutzen wollten um die neuen Tweets zu generieren. Es besteht aus einer "Embedding Layer" mit 50 Neuronen, zwei "LSTM Layer" mit je 100 Neuronen und zwei "Fully-Connected Layer", die erste mit 100 Neuronen und die zweite mit so vielen Neuronen wie es Wörter in unserem Vokabular gibt. Bei der Struktur des Models haben wir uns an einem Tutorial orientiert, das ein Word-Level Model in Keras zeigt (https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/). Weitere Erläuterungen und Problembeschreibungen sind als Kommentare im Code eingefügt.

In [1]:
# Importiere die benötigten packages
import numpy as np
import tensorflow as tf
import random
import scipy.io
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import re
# zusätzliche Pakete für eine weitere Methode zu tokenisieren
import nltk
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

### Hilfsfunktion

In [3]:
''' Diese Funktion entfernt alle Zeichen, die wir nicht haben wollen aus der Textdatei. Da die Tweets zunächst als Listenelemente
    aneinander gereit wurden (siehe Preprocessing), sind in der Textdatei noch sehr viele eckige Klammern. Außerdem wurden 
    Zeilenumbrüche als '/n' codiert und Tweets, die die Standardlänge von 140 Zeichen überschritten haben, wurden abgeschnitten,
    sodass teilweise unvollständige Worte vorkommen. Zudem gibt es noch einige weitere ungewünschte Zeichen, wie URL-links etc.
    Input:  Eine String Datei
    Output: Die Datei ohne die unerwünschten Zeichen
'''
def clean_text(text):
    
    # entfernt URL Links
    text = re.sub(r"http\S+", "", text)
    # entfernt @screen_name
    text = re.sub(r"@\S+", "", text)
    # entfernt "RT" für Re-Tweet
    text = re.sub(r"RT", "", text)
    # entfernt Sondersymbole wie Emojis
    text = re.sub(r"🇩🇪", "", text)
    #text = re.sub(r"😁", "", text)
    #text = re.sub(r"😀", "", text)
    #text = re.sub(r"'❤'", "", text)
    #text = re.sub(r"❤", "", text)
    #text = re.sub(r"🎉", "", text)
    # entfernt Sonderzeichen
    text = re.sub(r"'️", "", text)
    text = re.sub(r"\\n", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"]\S+", "", text)
    text = re.sub(r"…", "", text)
    text = re.sub(r". . . ", "", text)
    text = re.sub(r"r'\S+", "", text)
    #text = re.sub(r"#", "# ", text)
    text = re.sub(r"&amp", "", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"”", "", text)
    text = re.sub(r"’", "", text)
    text = re.sub(r"•", "", text)
    #text = re.sub(r"+", "", text)
    text = re.sub(r"»", "", text)
    text = re.sub(r"|", "", text)
    text = re.sub(r"«", "", text)
    text = re.sub(r"%", "", text)
    text = ''.join( c for c in text if  c not in '],[,/,:,&,_,1,2,3,4,5,6,7,8,9,0,\,&,;,-,),(,?,!')
   
    # wir möchten alle Buchstaben klein machen, um das Vokabular zu verringern
    text = text.lower()
    
    # ersetzt Umlaute und ß
    text = re.sub(r"ü", "ue", text)
    text = re.sub(r"ö", "oe", text)
    text = re.sub(r"ä", "ae", text)
    text = re.sub(r"ß", "ss", text)
    
    # setzt ein Leerzeichen vor jeden Punkt, sodass er als einzelnes Symbol gesehen wird und Wörter mit Punkt
    # nicht als eigenständige Wörter ins Vokabular eingehen
    text = re.sub(r"\.", " .", text)
    
    return text

### Einlesen der Daten

In [34]:
# Wähle ein Partei aus und kommentiere die restlichen Einlesefuntionen aus!

# Lade die Textdatei ins Notebook. Der Text wird utf-8 encoded.
# Zeichen, die von dieser Codierung nicht verstanden werden, werden erst einmal ignoriert.

# with open("tweets_all_parties/GRUENE.txt", encoding="utf8", errors='ignore') as f:
#     text = f.read()
# input_text = clean_text(text)

# with open("tweets_all_parties/LINKE.txt", encoding="utf8", errors='ignore') as f:
#     text = f.read()  
# input_text = clean_text(text)

with open("tweets_all_parties/AFD.txt", encoding="utf8", errors='ignore') as f:
    text = f.read()  
input_text = clean_text(text)

# with open("tweets_all_parties/FDP.txt", encoding="utf8", errors='ignore') as f:
#     text = f.read()  
# input_text = clean_text(text)

# with open("tweets_all_parties/SPD.txt", encoding="utf8", errors='ignore') as f:
#     text = f.read()  
# input_text = clean_text(text)

# with open("tweets_all_parties/CDU.txt", encoding="utf8", errors='ignore') as f:
#     text = f.read()  
# input_text = clean_text(text)

In [35]:
# Wir haben verschiede Versionen ausprobiert um den Text zu tokenisieren
# Sie haben alle ihre Vor- und Nachteile, manche beschleunigen das Training, aber dafür streichen sie alle Sonderzeichen,
# andere wiederum sind langsamer, weil alle Sonderzeichen als einzelne "Wörter" interpretiert werden etc
# Wir haben uns für die 4. Version entschieden, da sie ein guter Mittelweg ist das Vokabular klein zu halten und trotzdem
# Smileys etc zu lernen.

# 1. Möglichkeit - hier bleiben keine Punkte drin
#tokenized_text = list(input_text.split(" "))
#vocab = set(tokenized_text)

# 2. Möglichkeit - einzelne Buchstaben und Satzzeichen sind weg
#tokenizer = RegexpTokenizer(r'\w+')
#tokenized_text = list(tokenizer.tokenize(text_char))
#vocab = set(tokenized_text)

# 3. Möglichkeit - hier werden alle Leerzeichen, Satzzeichen ets als einzelne "Wörter" interpretiert
# dadurch ist dieser Ansatz sehr langsam
#p2 = re.compile(r'(\W+)')
#tokenized_text = p2.split(input_text)

# 4. Möglichkeit - behält smileys, Leerzeichen werden manuell am Ende wieder eingefügt
vocab = TreebankWordTokenizer()
tokenized_text = vocab.tokenize(input_text)

vocab = set(tokenized_text)
vocab_size = len(vocab)

print("vocabulary size: {}".format(vocab_size))

text_length = len(tokenized_text)
print("text length: {}".format(text_length))

vocabulary size: 1795
text length: 5763


In [36]:
# Kreiere Dictionaries um den Text in Zahlen umwandeln zu können 
# und später die Zahlen wieder in Text
# Gehe einmal durch ganze Vokabular und "zähle mit", sodass jedem Wort eine Zahl zugeordnet wird
word_to_id = {word:i for i, word in enumerate(vocab)}
id_to_word = {i:word for i, word in enumerate(vocab)}

# Übersetze den Text zu den zugehörigen IDs
text_idx = [word_to_id[word] for word in tokenized_text]


### Generieren des Datensets

In [37]:
# Bestimme die Länge der einzelnen Sequenzen.
# Wir haben uns erstmal für eine Länge von 10 Wörtern entschieden, da eine längere Abhängigkeit von Wörtern in dem Kontext von
# Tweets unserer Meinung nach wenig Sinn macht
# Zu jeder Sequenz gehört ein Targetwort, welches das nächste Wort in der Reihe ist. Daher addiert man zu der Länge noch 1
seq_len = 20+1
# Erstelle eine leere Liste zum abspeichern der Sequenzen
sequences = []
# Starte vorne in der Liste und speicher die ersten 11 IDs als erste Sequenz, gehe zur zweiten ID usw.
# Die letzte Sequenz endet mit dem letzten Wort als Target, dh das Wort an Stelle text_length - seq_len ist der Beginn der letzten Sequenz
for i in range(seq_len, len(text_idx)):
    # wähle die Sequenz von IDs aus
    seq = text_idx[i-seq_len:i]
    # und speicher sie in der Liste ab
    sequences.append(seq)
print('Total Sequences: {}'.format(len(sequences)))

Total Sequences: 5742


In [38]:
# Zerteile die Sequenzen in Inputs der Länge seq_len und Target der Länge 1
# Dazu mache die Liste erstmal zu einem Array
sequences = np.array(sequences)
# Nun zerteile die einzelnen Sequenzen
input_data, target = sequences[:,:-1], sequences[:,-1]

# Speicher die seq_length nocheinal dynamisch als die Länge der einzelnen Input Sequenzen ab, damit sie sich automatisch ändert,
# wenn der Input sich ändert
seq_length = input_data.shape[1]

## Das Model

### Vorbereitungen

In [39]:
# Jetzt fangen wir an das Model zu bauen
# Dafür brauchen wir ersteinmal ein Tensor Datenset

# Wie immer setzen wir zunächst den Grafen zurück
tf.reset_default_graph()

# Der Targetvektor wird one-hot encoded, damit er später mit dem Output der Softmax Funktion verglichen werden kann
# Dafür muss er zunächst zu dem richtigen Datentyp gecastet werden. Das ist hier kein Problem, weil die IDs alle ganze Zahlen sind
# und damit leicht als Integer statt als Float gespeichert werden können
target = tf.cast(target, dtype = tf.int32)
target = tf.one_hot(target, depth=vocab_size)
# Der Input wird ebenfalls als Integer gespeichert, aber nicht als one-hot
input_data = tf.cast(input_data, dtype = tf.int32)
print(input_data)
print(target)

# Kreiere das Tensorflow Datenset mit den Input Sequenzen und dem one-hot Target
dataset = tf.data.Dataset.from_tensor_slices((input_data,target))
# Shuffle die Daten
dataset = dataset.shuffle(buffer_size=len(sequences), reshuffle_each_iteration=True)

# Wähle ein Batchsize und zerteile das Datenset in Batches
batchsize = 128
dataset = dataset.batch(batchsize, drop_remainder=True)

# Initialisiere den Iterator wie gehabt
iterator = tf.data.Iterator.from_structure(dataset.output_types,dataset.output_shapes)
iterator_init_op = iterator.make_initializer(dataset)

# Generiere den Input Batch und den zugehörigen Target Batch mit Hilfe von .get_next()
next_batch = iterator.get_next()
input_data = next_batch[0]
target_data = next_batch[1]


# Initialisiere die Placeholder für den state der zwei LSTM layer
# Der initial_state soll zunächst aus Nullen bestehen und dann immer wieder an das Model zurück gegeben werden
# Dies passiert später im sess.run() mit der Hilfe von feed.dict{}
# Wähle die Größe der LSTM layer
lstm_size = 100
state_placeholder1 = tf.placeholder(tf.float32, [2, batchsize, lstm_size])
state_placeholder2 = tf.placeholder(tf.float32, [2, batchsize, lstm_size])

Tensor("Cast_1/x:0", shape=(5742, 20), dtype=int32)
Tensor("one_hot:0", shape=(5742, 1795), dtype=float32)


### Architektur

In [40]:
# Definiere das Model
# Zunächst wollen wir eine Embedding Layer, die die Embedding Vektoren der Input Wörter zurück gibt und die Embedding Matrix trainiert
with tf.variable_scope("embedding", reuse=tf.AUTO_REUSE):
    embedding = tf.contrib.layers.embed_sequence(
        ids = input_data,
        vocab_size = vocab_size,
        embed_dim = 100,
        unique = False,
        trainable = True)
    print(embedding)

# Danach folgen zwei LSTM Layer mit jeweils 100 Neuronen
with tf.variable_scope("LSTM1", reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.LSTMCell(
        num_units = lstm_size,
        forget_bias=1.0,
        state_is_tuple=True,
        activation='tanh')
    # der initial_state ist ein Tupel aus dem hidden state und dem cell state, beide werden gemeinsam durch den state_placeholder
    # eingelesen und dann zu einem LSTMStateTupel gemacht
    init_state = tf.nn.rnn_cell.LSTMStateTuple(state_placeholder1[0], state_placeholder1[1])
    outputs1, states1 = tf.nn.dynamic_rnn(cell = cell, inputs = embedding, initial_state = init_state)
    # wir wollen uns den aktuellen state merken um ihn in den nächsten batch zu geben
    remember1 = states1
    print(outputs1)
    
# Die zweite LSTM Layer ist aufgebaut wie die erste
with tf.variable_scope("LSTM2", reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.LSTMCell(
        num_units = lstm_size,
        num_proj = None,
        forget_bias=1.0,
        state_is_tuple=True,
        activation='tanh')
    init_state = tf.nn.rnn_cell.LSTMStateTuple(state_placeholder2[0],state_placeholder2[1])
    outputs2, states2 = tf.nn.dynamic_rnn(cell = cell, inputs = outputs1, initial_state = init_state)
    # wieder merken wir uns den state 
    remember2 = states2
    # nach den LSTM Layern wollen wir nun nicht mehr den Output für jedes Wort in der Sequenz weiter geben
    # sondern nur den Output des letzten Wortes in jeder Sequenz, weil wir nur das als prediction für das nächste Wort benötigen
    # und daraus den Fehler zu dem tatsächlichen letzten Wort berechnen können
    # um an den letzten Output zu kommen verändern wir zunächste die Reihenfolde der drei Dimensionen des Outputs
    # von (batchsize, seq_length, lstm_size) zu (seq_length, batchsize, lstm_size) damit für tf gather über die seq_length 
    # anwenden können
    temp = tf.transpose(outputs2, [1,0,2])
    last_output = tf.gather(temp, int(temp.get_shape()[0])-1)
    print(last_output)
    

# Die erste fully-connected layer besteht aus 100 Neuronen und benutzt die ReLU als activation function    
with tf.variable_scope("fully_connected1", reuse=tf.AUTO_REUSE):
    full = tf.contrib.layers.fully_connected(
        last_output,
        100,
        activation_fn = tf.nn.relu)
    print(full)

# Die zweite fully-connected layer hat so viele Neuronen, wie es Wörter in dem aktuellen Vokabular gibt.
# Die activation function ist nun die Softmax, sodass wir den Output des Models mit dem Target vergleichen können
with tf.variable_scope("fully_connected2", reuse=tf.AUTO_REUSE):
    # die logits werden für die Berechnung des Fehlers benutzt
    logits = tf.contrib.layers.fully_connected(
        full,
        vocab_size,
        activation_fn = None)
    # out wird für die prediction des nächsten Wortes benutzt
    out = tf.nn.softmax(logits)
    print(logits)

Tensor("embedding/EmbedSequence/embedding_lookup/Identity:0", shape=(128, 20, 100), dtype=float32)
Tensor("LSTM1/rnn/transpose_1:0", shape=(128, 20, 100), dtype=float32)
Tensor("LSTM2/GatherV2:0", shape=(128, 100), dtype=float32)
Tensor("fully_connected1/fully_connected/Relu:0", shape=(128, 100), dtype=float32)
Tensor("fully_connected2/fully_connected/BiasAdd:0", shape=(128, 1795), dtype=float32)


### Definition von Training Loss und der Optimierungsstrategie

In [41]:
# Der loss wird durch die cross entropy bestimmt, in der der Output des Models mit dem tatsächlichen Target verglichen wird
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=target_data, logits=logits))

In [42]:
# Zum Optimieren benutzen wir den Adams Optimizer und eine Lernrate von 0.001
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
training_step = optimizer.minimize(loss)

## Session Nr.1

In [ ]:
with tf.Session() as sess:
    # wir starten damit die Variablen zu initialisieren
    sess.run(tf.global_variables_initializer())
    
    # der "saver" speichert das fertig trainierte Model ab, damit es später an jedem beliebigen Rechner wieder eingelesen werden kann
    saver = tf.train.Saver()
    
    # bestimme die Anzahl der Epochen
    for epoch in range(180):
    
        # initialisiere einen Step counter für den Saver
        global_step = 0
        # lade das Datenset in den Iterator
        sess.run(iterator_init_op)
        
        # am Anfang jeder neuen Epoche sollen die Placeholder für den jeweiligen state der beiden LSTM Layer mit Nullen initialisiert werden
        # in jedem weiteren Step innerhalb der Epoche werden die states dann immer aufgerufen und weiter gegeben
        state1 = np.zeros([2, batchsize, lstm_size], dtype = np.float32)
        state2 = np.zeros([2, batchsize, lstm_size], dtype = np.float32)

        # gehe durch das Datenset bis es leer ist
        while True:
            try:

                # Starte das Training und gebe den loss aus
                # außerdem gebe die states der beiden LSTM Layer aus um sie für den nächsten Batch zu benutzen
                loss_val, state1, state2, _ = sess.run([loss, remember1, remember2, training_step],
                                                      feed_dict = {state_placeholder1:state1, state_placeholder2:state2})
                
                # Erhöhe den Global Step Count um 1
                global_step += 1

            # Stoppt sobald der Iterator leer ist
            except tf.errors.OutOfRangeError:
                break
        
        # Hier wird das Model abgespeichert 
        saver.save(sess, "./checkpointAFD_LSTM/model.ckpt", global_step = global_step)
                
       # Nach jeder Epoche soll der loss ausgegeben werden um den Trainingsprozess zu überwachen
        print("Epoch: {}, Loss: {:f}".format(epoch, loss_val))
        
        
        # Wir wollen schon während des Trainings Tweets generieren um zu schauen, wie sich die Qualität verändert
        
        # Bestimme die Länge der generierten Tweets
        tweet_length = 20
        
        # Als Startsequenz fürs Sampling können wir nicht wie bisher einfach eine zufällige Sequenz wählen,
        # da unser Model einen Input in Batches erwartet
        # deshalb muss auch der Sampling Input ein Batch sein
        # wir haben das so gelöst, dass wir einen Batch gebaut haben, der nur aus Sequenzen mit Nullen besteht und erst an 
        # der letzten Stelle die zufällige Sequenz hat
        
        # dazu wählen wir zunächst zufällig eine Startsequenz aus
        start = random.randint(0, len(text_idx) - seq_length)
        # und machen sie zu einem Tensor
        random_sequence = tf.constant(text_idx[start:start + seq_length], shape = (1,seq_length), dtype = tf.int32)
        # außerdem brauchen wir noch batchsize-1 leere Sequenzen
        seq_template = tf.zeros([batchsize-1,seq_length], dtype=tf.int32)
        # die leeren und die eine zufällige Sequenz packen wir dann zusammen für unseren Input
        random_seq_batch = tf.concat([seq_template, random_sequence], axis = 0)
        
        # Erstelle eine leere Liste um die generierten Wörter abzuspeichern
        sampled_words = []
        
        # Generiere Wort für Wort einen euen Tweet
        for n in range(tweet_length):
            
            # Da wir die generierten Tweets nur anschauen wollen und nicht mit einem Target vergleichen, brauchen wir Faketargets
            fake_target = tf.zeros([batchsize, vocab_size], dtype=tf.float32)
            sample_dataset = tf.data.Dataset.from_tensor_slices(([random_seq_batch], [fake_target]))
            # Lade das Sample Datenset in den Iterator
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Lese den Softmax Output der letzten Layer aus (dieser liefert die Wahrscheinlichkeiten für das nächste generierte Wort)
            # und gebe den state der LSTM Layer immer wieder neu rein
            sample_output, state1, state2 = sess.run([out, remember1, remember2],
                                                     feed_dict={state_placeholder1: state1, state_placeholder2:state2})

            # Wir wollen nur den letzten Output, da dieser von unserer random Startsequenz generiert wurde
            last = sample_output[-1,:]
            # Wir wählen ein Wort "zufällig" aber mit der Wahrscheinlichkeit p, die uns unser Output gibt,
            # aus unserem Vokabular aus...
            sample = np.random.choice(range(vocab_size), p=last.ravel())
            # ...und hängen es an die Liste der generierten Wörter an
            sampled_words.append(sample)
            
            # Update die Startsequenz mit dem neuen Wort um das nächste Wort zu generieren
            # Dies ist wieder etwas komplizierter, weil wir zunächst aus dem Batch die letzte Sequenz haben wollen
            # also die, die nicht aus Nullen besteht
            temp = random_seq_batch[batchsize-1,:]
            # an diese hängen wir dann das neue Wort dran und schneiden das erste ab
            next_random_sequence = tf.concat([temp[1:], tf.constant(sample,shape = (1,), dtype = tf.int32)], axis = 0)
            # dann muss sie noch in die richtige Form gebracht werden
            next_random_sequence = tf.reshape(next_random_sequence, shape = [1,seq_length])
            # bevor sie wieder mit den Nuller-Sequenzen zu einem Batch verbunden werden kann und als nächsten Input dient
            random_sequence = tf.concat([seq_template, next_random_sequence], axis = 0)
      
        # Zeige die generierten Tweets an
        # da wir in dieser Tokenisierung keine Leerzeichen haben, muss zwischen jedes Wort eins gesetzt werden
        sample_txt = ' '.join(id_to_word[idx] for idx in sampled_words)
        print('\n\n %s \n\n' % (sample_txt,))            

Epoch: 0, Loss: 6.132317


 jetzterstrecht sharia man soll ️⃣🔹🇧️🇮️🇱️🇩️🇺️🇳️🇬️ℹeckpfeiler stellen # . .➡der lesenswert buerger sagen allen freie darauf schicksalswahl afdwahlkampfauftakt dann verhalten kanzlercheck 


Epoch: 1, Loss: 5.926611


 lagerarbeiter # # frage stinkt # # folgt . dem ihren .📢 nrw eigene sonntag stimmun jetzterstrecht wurzeln mit seehofer 


Epoch: 2, Loss: 5.773694


 hat bestimmt traudichdeutschland # # stefan wir rayk doch otte defendeurope afd in . baeuerlichen wollte groko aber koran zukunft 


Epoch: 3, Loss: 6.053644


 ist von gham das afd herzliches willkommen btwdas sieht vorstandsmitglied traudichdeutschland die antiafd traudichdeutschlan retweete . afd traudichdeutschland mehr dass 


Epoch: 4, Loss: 5.888480
